In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

START = Int(0)
FREE = Int(1) # No modo Free  não existe qualquer força de travagem
STOPPING = Int(2)
BLOCKED = Int(3) # no modo Blocked as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa)
STOPPED = Int(4) # no modo Stopped o veículo está imobilizado.


#A componente contínua  do autómato usa variáveis contínuas $$\,V,v\,$$ para descrever a 
#velocidade do corpo   e a velocidade linear das rodas  ambas em relação so solo

#F força de travagem (Variavel)
#t Força de atrito a o solo (constante)

'''
1.
Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se 
desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas. 

2.
No contacto rodas/solo o atrito é constante porque a força de compressão é o peso; tem-se f = aP a, a constante de
atrito e P o peso. Ambos são fixos e independentes do modo.

3.
No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à 
diferença de velocidades F =  c(V-v) A  constante de proporcionalidade c depende do modo: é elevada no modo 
`Stopping` e baixa nos outros.


4. As  equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`, (V = -F) ^ (v = -aP +F)
e no modo `Blocked` (V = v) ^ (v = -aP)

5.Tanto no modo Blocked  como no modo Free  existe um “timer” que impede que o controlo aí permaneça mais do que 
t segundos.

6. Todos os “switchs” devem ser construídos de  modo a impedir a existência de trajetórias de Zenão.
                                                        
7. No instante inicial assume-se V = v = V0  ,  em que a velocidade V0 é o “input” do problema.( Isto é, sequências 
infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.)

In [ ]:
def gera_traco(declare,init,trans,k):
    with Solver(name="z3") as s:
        
        # completar
        trace = [declare(i) for i in range(k)]
        
        #semantics
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
            
        if s.solve():
            '''
            m = s.get_model()
            for n, v in m:
                print(f'{n} = {v}')
            '''
            for i in range(k):
                print("Passo ",i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("------------------------")
    
                        
####
gera_traco(declare,init,trans,5)